# Flight Price Predictor

In [1]:
!pip install scikit-learn -q
!pip install itertools -q

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools


In [2]:
# Load libraries
import pandas as pd # datahandling
import numpy as np # manipulating data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import itertools # memory efficiient way of manipulating iterators (sequences of data)
import matplotlib.pyplot as plt  # Plotting library
from tqdm import tqdm_notebook
import torch
from sklearn.model_selection import train_test_split


## Preprocessing

In [3]:
# Load dataset
df = pd.read_csv("https://raw.githubusercontent.com/imads20/BDS23/main/M1_Final_Assignment/business.csv")

# Print the first five rows:
df.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [4]:
# Print columns names and types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93487 entries, 0 to 93486
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        93487 non-null  object
 1   airline     93487 non-null  object
 2   ch_code     93487 non-null  object
 3   num_code    93487 non-null  int64 
 4   dep_time    93487 non-null  object
 5   from        93487 non-null  object
 6   time_taken  93487 non-null  object
 7   stop        93487 non-null  object
 8   arr_time    93487 non-null  object
 9   to          93487 non-null  object
 10  price       93487 non-null  object
dtypes: int64(1), object(10)
memory usage: 7.8+ MB


In [5]:
# Convert time_taken to number of minutes taken

# From the head() above, we can see that time_taken is formatted like '02h 00m'.
# Converting this format to number of minutes is a bit tedious, so we build a function to help us

# Function for converting the hour-minute format to minutes
def convert_to_minutes(time_str):
    hours, minutes_str = time_str.split('h ')   # Take the time string and split it by 'h '.
                                                # This will split the string into two parts:
                                                # The first part will correspond to the hours number of hours
                                                # The second part will be the number of minutes and an 'm'

    minutes = minutes_str.replace("m", "") # Replace the 'm' in the minutes with nothing

    hours = int(hours) # Convert hours and minutes to integers to ensure that we have numeric format
    minutes = int(minutes)

    total_minutes = (hours * 60) + minutes # Find the total number of minutes

    return total_minutes

# Apply the function defined above to the column
df['duration_minutes'] = df['time_taken'].apply(convert_to_minutes)
df['duration_minutes'][:5] # Print the first 5 rows as control

0     120
1     135
2    1485
3    1590
4     400
Name: duration_minutes, dtype: int64

In [6]:
# Convert stop to number of stops

# Looking at the head() above, the stop column seems to be defined in an interesting manner.
# In order to get an overview of the formatting across the whole column, we can print the unique values in the column:
df['stop'].unique()

array(['non-stop ',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXU\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Chennai\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Lucknow\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia STV\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Hyderabad\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia GAY\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '2+-stop',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Guwahati\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia GAU\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia VTZ\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia NDC\n\t\t\t\t\t\t\t\t\t\t\t\t',
       '1-stop\n\t\t\t\t\t\t\t\t\t\t\t

In [7]:
# The columns contains the number of stops as well as the location of the stop.
# The number of stops can either be zero, one, or more than two.
# Once again, cleaning up this format is a bit tedious, so we build a function to help us

# Function for defining number of stops
def number_of_stops(string):
    stops_str = string.split('-stop')[0]    # Split the string by '-stop' and keep only the first part of the split
                                            # When doing this we will be left with either 'non', '1' or '2+' values

    # Define what the number of stop is based on the string
    if stops_str == 'non':
        stops = 0
    elif stops_str == '1':
        stops = 1
    elif stops_str == '2+':
        stops = 2
    else:
        stops = np.nan  # If the format of the sting doesn't match the above, set it as nan

    return stops

# Apply the function defined above to the column
df['stop'] = df['stop'].apply(number_of_stops)
df['stop'][:5] # Control

0    0
1    0
2    1
3    1
4    1
Name: stop, dtype: int64

In [8]:
# Convert price to numeical value:

# The head() above shows that price contains a ','.
# In order to convert the price to a numerical value, we need to remove this
df['price'] = df['price'].str.replace(",", "")

# Afterwards, convert the string to an integer to ensure that the column gets interpreted as a numeric value
df['price'] = df['price'].astype(int)
df['price'][:5] # Control

0    25612
1    25612
2    42220
3    44450
4    46690
Name: price, dtype: int64

### Handling missing values

In [9]:
# Find the number of missing values in each column
df.isnull().sum()

date                0
airline             0
ch_code             0
num_code            0
dep_time            0
from                0
time_taken          0
stop                0
arr_time            0
to                  0
price               0
duration_minutes    0
dtype: int64

In [10]:
# We remove any duplicate values in the data set
df = df.drop_duplicates()
df.info()
# We have same number of observations so there were no duplicate values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93487 entries, 0 to 93486
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              93487 non-null  object
 1   airline           93487 non-null  object
 2   ch_code           93487 non-null  object
 3   num_code          93487 non-null  int64 
 4   dep_time          93487 non-null  object
 5   from              93487 non-null  object
 6   time_taken        93487 non-null  object
 7   stop              93487 non-null  int64 
 8   arr_time          93487 non-null  object
 9   to                93487 non-null  object
 10  price             93487 non-null  int64 
 11  duration_minutes  93487 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 9.3+ MB


### Feature engineering

In [11]:
# In order to make a distance column, we to find all the unique cities in 'from' and 'to':
print("Unique cities to travel from " + str(list(df['from'].unique())))
print("Unique cities to travel to " + str(list(df['to'].unique())))

Unique cities to travel from ['Delhi', 'Mumbai', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai']
Unique cities to travel to ['Mumbai', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai', 'Delhi']


In [12]:
# From the above we can see that we have 6 unique cities to travel from and to.
# This means that we overall have 15 different combinations of routes (assuming that you cannot travel to the same place that you took off from).

# The next thing to do is to define a list with distance between the 15 different combinations.
# The distances between the cities has been found by looking them up on the internet.
city_distances = {
    ('Delhi', 'Mumbai'): 1139,
    ('Delhi', 'Bangalore'): 1710,
    ('Delhi', 'Kolkata'): 1313,
    ('Delhi', 'Hyderabad'): 1268,
    ('Delhi', 'Chennai'): 1761,
    ('Mumbai', 'Bangalore'): 835,
    ('Mumbai', 'Kolkata'): 1666,
    ('Mumbai', 'Hyderabad'): 624,
    ('Mumbai', 'Chennai'): 1034,
    ('Bangalore', 'Kolkata'): 1547,
    ('Bangalore', 'Hyderabad'): 455,
    ('Bangalore', 'Chennai'): 268,
    ('Kolkata', 'Hyderabad'): 1208,
    ('Kolkata', 'Chennai'): 1386,
    ('Hyderabad', 'Chennai'): 507
}

# Function for finding distance between 'to' and 'from' columns for each row in our dataframe
def calculate_distance(row):
    city1, city2 = row['from'], row['to'] # Define city1 and city2 based on from and to columns

    distance = city_distances.get((city1, city2),                       # Get the distance between the cities regardless of the order of to and from.
                                  city_distances.get((city2, city1),    # This is done by first trying to get the distance between city1 and city2 combo.
                                                     None))             # If this combo doesn't exist in the city_distances list, we try the city2 and city 1.
                                                                        # If this doesn't exist in the list, the distance should be None (indicating that the route
                                                                        # is missiing in the list).

    return distance

# Create a new column called 'distance' using the above function.
df['distance'] = df.apply(calculate_distance, axis=1)
df.head() # Control

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,duration_minutes,distance
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,0,20:00,Mumbai,25612,120,1139
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,0,21:15,Mumbai,25612,135,1139
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1,20:45,Mumbai,42220,1485,1139
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1,23:55,Mumbai,44450,1590,1139
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1,23:55,Mumbai,46690,400,1139


In [13]:
# Finally, we create bins for the departure time and the arrival time.
# We do this is generalize the columns such that we only have a few distics groups and not every possible combination of every hour and every minute of the day.
# By generalising the columns into bins, it will be easier to cluster the data based on the time of the day.
# Likewise, it will also be easier to predict the price based on the time of the day.

# First we need to define the bins that we will use for the time of the day.
# We have decided on the following:
    #Night: 23:00-03:59
    #Early_morning: 04:00-06:59
    #Morning: 07:00-10:59
    #Midday: 11:00-13:59
    #Afternoon: 14:00-16:59
    #Evening: 17:00-19:59
    #Late_evening: 20:00-22:59


# Function for grouping into a bin the time based on the above system
def group_time(time):
    group = None

    if time < '04:00':
        group = 'Night'
    elif time >= '04:00' and time < '07:00':
        group = 'Early_morning'
    elif time >= '07:00' and time < '11:00':
        group = 'Morning'
    elif time >= '11:00' and time < '14:00':
        group = 'Midday'
    elif time >= '14:00' and time < '17:00':
        group = 'Afternoon'
    elif time >= '17:00' and time < '20:00':
        group = 'Evening'
    elif time >= '20:00' and time < '23:00':
        group = 'Late_evening'
    elif time >= '23:00':
        group = 'Night'

    return group

# Apply the function to both 'dep_time' and 'arr_time' columns:
df['dep_time'] = df['dep_time'].apply(group_time)
df['arr_time'] = df['arr_time'].apply(group_time)
df.head() # Control

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,duration_minutes,distance
0,11-02-2022,Air India,AI,868,Evening,Delhi,02h 00m,0,Late_evening,Mumbai,25612,120,1139
1,11-02-2022,Air India,AI,624,Evening,Delhi,02h 15m,0,Late_evening,Mumbai,25612,135,1139
2,11-02-2022,Air India,AI,531,Late_evening,Delhi,24h 45m,1,Late_evening,Mumbai,42220,1485,1139
3,11-02-2022,Air India,AI,839,Late_evening,Delhi,26h 30m,1,Night,Mumbai,44450,1590,1139
4,11-02-2022,Air India,AI,544,Evening,Delhi,06h 40m,1,Night,Mumbai,46690,400,1139


In [14]:
# From the above, we can see that we have quite a few columns in our dataframe.
# Given the content of some of the columns, we will not be using all of them.
# For example the 'ch_code' and 'num_code' are likely not any big indicators of the price
# The 'date' could have played a role in the price determination if we had more data (e.g., we would expect higher flight prices during hollidays
    # or that prices would have an increasing trend over time due to inflation) but given that we only have 50 days of data, it's not enough to
    # spot a seasonality pattern or a trend pattern.

# So we overrwrite our dataframe such that we only keep the columns we're going to use
df = df[['price', 'airline', 'from', 'to', 'dep_time', 'arr_time', 'distance', 'duration_minutes', 'stop']]
df.head()

,price,airline,from,to,dep_time,arr_time,distance,duration_minutes,stop
0,25612,Air India,Delhi,Mumbai,Evening,Late_evening,1139,120,0
1,25612,Air India,Delhi,Mumbai,Evening,Late_evening,1139,135,0
2,42220,Air India,Delhi,Mumbai,Late_evening,Late_evening,1139,1485,1
3,44450,Air India,Delhi,Mumbai,Late_evening,Night,1139,1590,1
4,46690,Air India,Delhi,Mumbai,Evening,Night,1139,400,1


### EDA

As this dataset have been using during M1 and the main focus not is the EDA, we abstrain from doing it in this assignment.

### Labelencoder, One Hot Encoder & standard scalar

In [15]:
# We only have two airlines, so we will make this into a dummy variable using LabelEncoder
le = LabelEncoder()
df['airline'] = le.fit_transform(df['airline'])
df.head() #Control

<ipython-input-15-8856aa357e2d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['airline'] = le.fit_transform(df['airline'])


,price,airline,from,to,dep_time,arr_time,distance,duration_minutes,stop
0,25612,0,Delhi,Mumbai,Evening,Late_evening,1139,120,0
1,25612,0,Delhi,Mumbai,Evening,Late_evening,1139,135,0
2,42220,0,Delhi,Mumbai,Late_evening,Late_evening,1139,1485,1
3,44450,0,Delhi,Mumbai,Late_evening,Night,1139,1590,1
4,46690,0,Delhi,Mumbai,Evening,Night,1139,400,1


In [16]:
# We will scale our numerical columns (price, airline, distance, duration_minutes, stop) using the StandardScaler
df_num = df[['price', 'airline', 'distance', 'duration_minutes', 'stop']]
scaler= MinMaxScaler()
scaled_values = scaler.fit_transform(df_num)
df_num=pd.DataFrame(scaled_values,columns=df_num.columns)
df_num.head() # Control

,price,airline,distance,duration_minutes,stop
0,0.122552,0.0,0.583389,0.021390,0.0
1,0.122552,0.0,0.583389,0.026738,0.0
2,0.272078,0.0,0.583389,0.508021,0.5
3,0.292155,0.0,0.583389,0.545455,0.5
4,0.312323,0.0,0.583389,0.121212,0.5


In [17]:
# We will encode our categorical columns (to, from, dep_time, arr_time) using OneHotEncoder to avoid any incorrect relationships
df_cat = df[['to', 'from', 'dep_time', 'arr_time']]

# To prevent columns with the same name, we add a suffix to the variables
df_cat['to'] = 'to_' + df_cat['to']
df_cat['from'] = 'from_' + df_cat['from']
df_cat['dep_time'] = 'dep_' + df_cat['dep_time']
df_cat['arr_time'] = 'arr_' + df_cat['arr_time']

# Apply the OneHotEncoder
ohe_X = OneHotEncoder(sparse=False)
X_ohe = ohe_X.fit_transform(df_cat.iloc[:,:])
columns_X_ohe = list(itertools.chain(*ohe_X.categories_)) # Get the column names
columns_X_ohe

<ipython-input-17-26964fd476e7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['to'] = 'to_' + df_cat['to']
<ipython-input-17-26964fd476e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['from'] = 'from_' + df_cat['from']
<ipython-input-17-26964fd476e7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

['to_Bangalore',
 'to_Chennai',
 'to_Delhi',
 'to_Hyderabad',
 'to_Kolkata',
 'to_Mumbai',
 'from_Bangalore',
 'from_Chennai',
 'from_Delhi',
 'from_Hyderabad',
 'from_Kolkata',
 'from_Mumbai',
 'dep_Afternoon',
 'dep_Early_morning',
 'dep_Evening',
 'dep_Late_evening',
 'dep_Midday',
 'dep_Morning',
 'dep_Night',
 'arr_Afternoon',
 'arr_Early_morning',
 'arr_Evening',
 'arr_Late_evening',
 'arr_Midday',
 'arr_Morning',
 'arr_Night']

In [18]:
df_cat = pd.DataFrame(X_ohe, columns = columns_X_ohe)

In [19]:
# Combine dataframe using pd.merge
df_combined = pd.merge(df_num, df_cat, left_index=True, right_index=True, how='left')
df_combined.head()

,price,airline,distance,duration_minutes,stop,to_Bangalore,to_Chennai,to_Delhi,to_Hyderabad,to_Kolkata,...,dep_Midday,dep_Morning,dep_Night,arr_Afternoon,arr_Early_morning,arr_Evening,arr_Late_evening,arr_Midday,arr_Morning,arr_Night
0,0.122552,0.0,0.583389,0.021390,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.122552,0.0,0.583389,0.026738,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.272078,0.0,0.583389,0.508021,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.292155,0.0,0.583389,0.545455,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.312323,0.0,0.583389,0.121212,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Prediction

In [20]:
df_combined.head()

,price,airline,distance,duration_minutes,stop,to_Bangalore,to_Chennai,to_Delhi,to_Hyderabad,to_Kolkata,...,dep_Midday,dep_Morning,dep_Night,arr_Afternoon,arr_Early_morning,arr_Evening,arr_Late_evening,arr_Midday,arr_Morning,arr_Night
0,0.122552,0.0,0.583389,0.021390,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.122552,0.0,0.583389,0.026738,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.272078,0.0,0.583389,0.508021,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.292155,0.0,0.583389,0.545455,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.312323,0.0,0.583389,0.121212,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
len(df_combined.columns)

31

In [22]:
# First we split the data into features (X) and target variable (y)
X = df_combined.drop(['price'],axis=1)
y = df_combined['price']

# We split data in train set and test set with the test set using 20% of the data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, # test set is 20% of the whole dataset
                                                    random_state=42) # we set a random state to always get the same sample

In [23]:
# Convert the pandas Series to a PyTorch tensor
tensor_data_Y = torch.tensor(y_train.values, dtype=torch.float32)
tensor_data_X = torch.tensor(X_train.values, dtype=torch.float32)
tensor_data_Y_test = torch.tensor(y_test.values, dtype=torch.float32)
tensor_data_X_test = torch.tensor(X_test.values, dtype=torch.float32)

In [24]:
tensor_data_X[0] #First row of X values

tensor([0.0000, 0.5834, 0.2513, 0.5000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000])

In [25]:
tensor_data_X.view(-1, len(df_combined.columns)-1).shape

torch.Size([74789, 30])

## Stochastic Gradient Descent Pytorch

In [29]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.05
# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Linear(len(df_combined.columns)-1,1, bias=False)
model_net1_actfun = torch.nn.Identity()
model_net1.weight.data.fill_(w)

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1_actfun.forward(model_net1.forward(tensor_data_X[i].reshape(-1)))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-29-663d786c2ad2>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0118
--------------------------------------------------


Epoch 2 Average Loss: 0.0093
--------------------------------------------------


Epoch 3 Average Loss: 0.0093
--------------------------------------------------



In [67]:
# Initializing Hyperparameters
epochs = 5
learning_rate = 0.01
# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Linear(len(df_combined.columns)-1,1, bias=False)
model_net1_actfun = torch.nn.Identity()
model_net1.weight.data.fill_(w)

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1_actfun.forward(model_net1.forward(tensor_data_X[i].reshape(-1)))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-67-e6e4a69da983>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0169
--------------------------------------------------


Epoch 2 Average Loss: 0.0071
--------------------------------------------------


Epoch 3 Average Loss: 0.0071
--------------------------------------------------


Epoch 4 Average Loss: 0.0071
--------------------------------------------------


Epoch 5 Average Loss: 0.0071
--------------------------------------------------



In [64]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.03
# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Linear(len(df_combined.columns)-1,1, bias=False)
model_net1_actfun = torch.nn.Identity()
model_net1.weight.data.fill_(w)

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1_actfun.forward(model_net1.forward(tensor_data_X[i].reshape(-1)))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-64-7c7e115534ea>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0117
--------------------------------------------------


Epoch 2 Average Loss: 0.0081
--------------------------------------------------


Epoch 3 Average Loss: 0.0081
--------------------------------------------------



In [65]:
# Initializing Hyperparameters
epochs = 5
learning_rate = 0.01

# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Linear(len(df_combined.columns)-1,1, bias=False)
model_net1_actfun = torch.nn.ReLU()
model_net1.weight.data.fill_(w)

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1_actfun.forward(model_net1.forward(tensor_data_X[i].reshape(-1)))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-65-963573af3407>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0205
--------------------------------------------------


Epoch 2 Average Loss: 0.0071
--------------------------------------------------


Epoch 3 Average Loss: 0.0071
--------------------------------------------------


Epoch 4 Average Loss: 0.0071
--------------------------------------------------


Epoch 5 Average Loss: 0.0071
--------------------------------------------------



In [66]:
# Initializing Hyperparameters
epochs = 5
learning_rate = 0.01

# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net1 = torch.nn.Linear(len(df_combined.columns)-1,1, bias=False)
model_net1_actfun = torch.nn.Tanh()
model_net1.weight.data.fill_(w)

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net1.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net1_actfun.forward(model_net1.forward(tensor_data_X[i].reshape(-1)))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-66-00b952232529>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.4171
--------------------------------------------------


Epoch 2 Average Loss: 0.4171
--------------------------------------------------


Epoch 3 Average Loss: 0.4171
--------------------------------------------------


Epoch 4 Average Loss: 0.4171
--------------------------------------------------


Epoch 5 Average Loss: 0.4170
--------------------------------------------------



As ReLU and identity seems to be equally good, they both will be uograded to contain 2 hidden layers.

## 2 hidden layers

In [69]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.01


# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.Linear(30,3),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(3,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-69-b89199e93736>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.1467
--------------------------------------------------


Epoch 2 Average Loss: 0.1467
--------------------------------------------------


Epoch 3 Average Loss: 0.1467
--------------------------------------------------



In [75]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.01


# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.ReLU(),
                                 torch.nn.Linear(30,3),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.ReLU(),
                                 torch.nn.Linear(3,1),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-75-fa9bb1b584ee>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0111
--------------------------------------------------


Epoch 2 Average Loss: 0.0084
--------------------------------------------------


Epoch 3 Average Loss: 0.0083
--------------------------------------------------



In [76]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.01


# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.ReLU(),
                                 torch.nn.Linear(30,15),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.ReLU(),
                                 torch.nn.Linear(15,3),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.ReLU(),
                                 torch.nn.Linear(3,1),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-76-c793274dffc2>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0120
--------------------------------------------------


Epoch 2 Average Loss: 0.0103
--------------------------------------------------


Epoch 3 Average Loss: 0.0093
--------------------------------------------------



In [77]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.01


# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.Linear(30,15),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(15,3),
                                 torch.nn.ReLU(),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.Linear(3,1),
                                 torch.nn.ReLU(),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-77-a2f526b0fbd5>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0129
--------------------------------------------------


Epoch 2 Average Loss: 0.0116
--------------------------------------------------


Epoch 3 Average Loss: 0.0110
--------------------------------------------------



## The chosen one ⚡️🧙🏼‍♂️

In [32]:
# Initializing Hyperparameters
epochs = 3
learning_rate = 0.01


# Initializing Parameters
w = 1

loss_set = {}

# 1. Creating a FeedForwardNetwork
# 1.1 Structure (Architecture) of NN
model_net2 = torch.nn.Sequential(torch.nn.ReLU(),
                                 torch.nn.Linear(30,3),
                                 torch.nn.Dropout(0.33),

                                 torch.nn.ReLU(),
                                 torch.nn.Linear(3,1),

                                 );

# 1.2 Loss Function
loss_mse = torch.nn.MSELoss()

# 1.3 Optmization Approch
optimizer = torch.optim.SGD(model_net2.parameters(), lr=learning_rate)

w_his = []
w_his.append(w)
# Loop over the number of epochs
for epoch in tqdm_notebook(range(epochs), desc="Epochs"):
    epoch_loss = 0.0

    # Loop over each sample in the dataset
    for i in range(tensor_data_X.size(0)):

      # 2. Forward Pass
      output = model_net2.forward(tensor_data_X[i].reshape(-1))

      # 3. FeedForward Evaluation
      loss = loss_mse(output, tensor_data_Y[i].reshape(-1))
      optimizer.zero_grad();

      # 4. Backward Pass / Gradient Calculation
      loss.backward()

      # Store the loss for each epoch
      epoch_loss += loss.item()

      # 5. Back Propagation / Update Weights
      optimizer.step()

      # Store the weight value for each sample of data
      w_his.append(float(model_net1.weight.data[0][0]))


    # Calculate and display average loss for the epoch
    epoch_loss /= tensor_data_X.size(0)

    # Store the loss for each sample of data
    loss_set[epoch] = epoch_loss
    print(f"\nEpoch {epoch+1} Average Loss: {epoch_loss:.4f}\n{'-'*50}\n")

<ipython-input-32-fa9bb1b584ee>:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs), desc="Epochs"):


Epochs:   0%|          | 0/3 [00:00<?, ?it/s]


Epoch 1 Average Loss: 0.0120
--------------------------------------------------


Epoch 2 Average Loss: 0.0094
--------------------------------------------------


Epoch 3 Average Loss: 0.0091
--------------------------------------------------



In [33]:
# Print the parameters of all layers
for name, param in model_net2.named_parameters():
    print(f"Layer: {name}")
    print(f"Size: {param.size()}")
    print(f"Values: \n{param.data}\n")

Layer: 1.weight
Size: torch.Size([3, 30])
Values: 
tensor([[-6.7764e-02, -8.3689e-02,  9.4034e-02, -4.1871e-02,  2.8062e-04,
          5.8136e-02,  4.9860e-02,  7.2849e-02, -1.3263e-01,  3.3953e-02,
          4.6024e-02,  1.2389e-02, -3.6366e-02,  5.5453e-02, -1.2149e-01,
         -3.1238e-02,  5.5956e-02, -9.9736e-02, -2.1783e-02, -2.3914e-02,
         -6.9232e-02, -9.7590e-02, -1.3229e-01, -1.5114e-01,  1.5752e-01,
          4.4050e-02,  3.1173e-02, -3.3934e-03,  3.2217e-02, -9.5905e-02],
        [-1.3156e-01,  7.5625e-02, -2.3674e-02, -6.4391e-01, -6.9512e-03,
          1.4137e-02,  1.0497e-01,  4.7359e-02,  6.6545e-04,  1.0560e-01,
         -2.7474e-04,  1.1441e-03,  1.1813e-01,  5.4756e-02,  2.3931e-03,
          1.1431e-01,  6.3331e-02,  4.5005e-02,  6.7199e-02,  5.5488e-02,
          7.6375e-02,  5.6198e-02, -1.0325e-01,  1.0770e-01,  8.3272e-02,
          9.1435e-02,  8.0042e-02,  8.5831e-02,  8.5279e-02,  9.7079e-02],
        [-6.1769e-02, -5.2627e-02,  1.3546e-02, -4.7539e-01

## Testing the flight prediction model on the test dataset

In [34]:
model_net2(tensor_data_X_test) #The predicted values

tensor([[0.3326],
        [0.3377],
        [0.3899],
        ...,
        [0.3352],
        [0.3372],
        [0.3588]], grad_fn=<AddmmBackward0>)

In [35]:
y_test #The actual values

91395    0.292146
69621    0.359005
43497    0.384295
58572    0.229817
36893    0.469727
           ...   
40444    0.405876
53930    0.290625
12148    0.408793
13330    0.377569
89059    0.289400
Name: price, Length: 18698, dtype: float64

In [36]:
tensor_data_Y_test-model_net2(tensor_data_X_test) #The absolute difference

tensor([[-0.1033, -0.0364, -0.0111,  ...,  0.0134, -0.0179, -0.1060],
        [-0.1033, -0.0364, -0.0111,  ...,  0.0134, -0.0179, -0.1060],
        [-0.0618,  0.0050,  0.0303,  ...,  0.0548,  0.0236, -0.0646],
        ...,
        [-0.1033, -0.0364, -0.0111,  ...,  0.0134, -0.0179, -0.1060],
        [-0.0451,  0.0218,  0.0471,  ...,  0.0716,  0.0404, -0.0478],
        [-0.0666,  0.0002,  0.0255,  ...,  0.0500,  0.0188, -0.0694]],
       grad_fn=<SubBackward0>)

In [37]:
(tensor_data_Y_test-model_net2(tensor_data_X_test))/tensor_data_Y_test*100 #The percentage difference

tensor([[-15.9111,   5.6753,  11.8827,  ...,  17.1634,  10.3131, -17.0109],
        [-19.3734,   2.8578,   9.2506,  ...,  14.6890,   7.6342, -20.5060],
        [-21.1605,   1.4035,   7.8920,  ...,  13.4118,   6.2514, -22.3101],
        ...,
        [-15.3313,   6.1471,  12.3235,  ...,  17.5777,  10.7617, -16.4256],
        [-21.7341,   0.9367,   7.4559,  ...,  13.0019,   5.8075, -22.8892],
        [-22.8005,   0.0689,   6.6453,  ...,  12.2398,   4.9824, -23.9657]],
       grad_fn=<MulBackward0>)

In [38]:
(tensor_data_Y_test-model_net2(tensor_data_X_test))**2

tensor([[1.6402e-03, 6.9478e-04, 2.6676e-03,  ..., 5.7983e-03, 2.0181e-03,
         1.8702e-03],
        [3.2034e-03, 1.0526e-04, 1.2638e-03,  ..., 3.6057e-03, 8.3084e-04,
         3.5218e-03],
        [3.8217e-03, 2.5387e-05, 9.1983e-04,  ..., 3.0059e-03, 5.5711e-04,
         4.1687e-03],
        ...,
        [2.3704e-04, 6.7657e-03, 1.1566e-02,  ..., 1.7435e-02, 1.0164e-02,
         1.6002e-04],
        [1.0668e-02, 1.3270e-03, 1.2406e-04,  ..., 1.7848e-04, 3.1912e-04,
         1.1243e-02],
        [4.4370e-03, 6.1157e-08, 6.5216e-04,  ..., 2.5035e-03, 3.5389e-04,
         4.8104e-03]], grad_fn=<PowBackward0>)

In [55]:
y_true=tensor_data_Y_test.numpy()
y_pred=(model_net2(tensor_data_X_test)).detach().numpy()

In [56]:
diff=(y_true-y_pred)

In [58]:
diff=diff**2

In [62]:
MSE=diff.mean() #MSE
MSE

0.019192884

In [64]:
RMSE=MSE**0.5 #RMSE
RMSE

0.13853838365173693